### Notebook to generate wind and heat flux forcing for idealized coastal shelf model. 
The heat fluxes are included for tests of the KPP vertical mixing scheme *(LMD, 1994)*

In [13]:
#Packages - too many for this notebook
%matplotlib inline
import numpy as np
import xgcm
from xgcm import Grid
import xarray as xr
import xroms
from datetime import datetime

import glob
from xhistogram.xarray import histogram
import cmocean.cm as cmo
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.ticker as tick
from matplotlib.dates import DateFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator

### To switch to KPP, we have to tag unique CPP flags, recompile the model, the modify the forcing to include some heat fluxes 
See link for why variables have specific names/units. https://www.myroms.org/index.php?page=forcing.
> - Time dimensions for each variable. Wind stress is ```sms_time```
> - Surface $u$ and $v$ momentum stress due to the winds. I prescribe this directly, rather than giving it a wind speed and having ROMS do the internal calculations
> - Surface net, shortwave, longwave, and sensitive heat fluxes. These all have to have unique time coordinates 

In [2]:
def make_oscillatory(ndays, dtw, wind_freq, u_amp, v_amp, output_path):
  
    t = xr.DataArray(np.arange(0, ndays+dtw, dtw)[:-1], dims=['sms_time'])
    t1 = xr.DataArray(np.arange(0, ndays+dtw, dtw)[:-1], dims=['srf_time'])
    t2 = xr.DataArray(np.arange(0, ndays+dtw, dtw)[:-1], dims=['lrf_time'])
    t3 = xr.DataArray(np.arange(0, ndays+dtw, dtw)[:-1], dims=['sen_time'])
    t4 = xr.DataArray(np.arange(0, ndays+dtw, dtw)[:-1], dims=['shw_time'])
    
    #Create some oscillatory wind stress of a specified frequency and amplitude.
    uwind_strs = xr.DataArray((np.sin(t*wind_freq*np.pi)*u_amp), dims=['sms_time'])
    vwind_strs = xr.DataArray((np.sin(t*wind_freq*np.pi)*v_amp), dims=['sms_time'])
    
    #Set the net heat flux to zero
    shflux_n = xr.DataArray((np.sin(t*wind_freq*np.pi)*0), dims=['shw_time'])
    sswflux_n = xr.DataArray((np.sin(t*wind_freq*np.pi)*0), dims=['srf_time'])
    slwflux_n = xr.DataArray((np.sin(t*wind_freq*np.pi)*0), dims=['lrf_time'])
    senflux_n = xr.DataArray((np.sin(t*wind_freq*np.pi)*0), dims=['sen_time'])
    
    # Create dataset
    ds = xr.Dataset({'sms_time': t, 'srf_time': t1, 'lrf_time': t2, 'sen_time': t3, 'shw_time': t4,
                     'sustr': uwind_strs, 'svstr': vwind_strs,
                     'shflux': shflux_n, 'swrad': sswflux_n, 'lwrad': slwflux_n, 'sensible': senflux_n,})
    # Metadata
    ds.attrs['Description'] = 'Forcing for ideal shelf'
    ds.attrs['Author'] = 'Dylan Schlichting'
    ds.attrs['Created'] = datetime.now().isoformat()
    ds.attrs['type'] = 'ROMS FRC file'
    # List variable attributes 
    ds['sms_time'].attrs['units'] = 'days'
    ds['srf_time'].attrs['units'] = 'days'
    ds['lrf_time'].attrs['units'] = 'days'
    ds['sen_time'].attrs['units'] = 'days'
    ds['shw_time'].attrs['units'] = 'days'
    ds['sustr'].attrs['units'] = 'Newton meter-2'
    ds['svstr'].attrs['units'] = 'Newton meter-2'
    ds['shflux'].attrs['units'] = 'Watts meter-2'
    ds['swrad'].attrs['units'] = 'Watts meter-2'
    ds['lwrad'].attrs['units'] = 'Watts meter-2'

    return ds

ndays = 60
wind_freq = 3
dtw = 1/72 
u_amp = 0.1
v_amp = 0 

output_path = 'shelf_dx_500_uwind_zeropoint1pa'+'_frc.nc'
ds = make_oscillatory(ndays, dtw, wind_freq, u_amp, v_amp, output_path)
ds.to_netcdf(output_path)

### Open the output and look at to make sure everything is all good. 

In [ ]:
dsf = xr.open_dataset(output_path)
dsf